In [3]:
import transformers
import torch, math, einops
from torch import nn

from torch.distributions.categorical import Categorical

from src.encoder import GPT2Encoder
from src.decoder import GPT2Decoder
from src.GPT2 import GPT2_Block, GPT2
from src import BlockGenerator
from src import Tokenizer, random_graph_maker

In [3]:
from datasets import load_dataset

train=load_dataset("wikipedia", "20220301.simple",split="train[:80%]")
valid=load_dataset("wikipedia", "20220301.simple",split="train[80%:]")

Found cached dataset wikipedia (/Users/francescosacco/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)
Found cached dataset wikipedia (/Users/francescosacco/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [4]:
tokenizer = Tokenizer('gpt2')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

dK = 64
dV = 64
heads = 12
d_Embedding = dK*heads

encoder = GPT2Encoder()
decoder = GPT2Decoder()
block_generator = BlockGenerator(GPT2_Block)
model = GPT2(tokenizer, encoder, block_generator, decoder)

#pretrained = transformers.GPT2LMHeadModel.from_pretrained('gpt2').to(device)
#model.load_from_original(pretrained)

graph_maker = random_graph_maker(50, 50)


In [5]:
from src.decoder import Loss, last_word_loss

loss_function=Loss(decoder, last_word_loss)
lr=1e-6
gamma=0.99

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)

In [6]:
n_epochs = 2
model.train()
losses = []
for _ in range(n_epochs):
    for page in train:
        text = page['text'][:3000]
        nodes = tokenizer(text)
        nodes, target = nodes[:-3], nodes[-3]
        edge_index = graph_maker(len(nodes))

        optimizer.zero_grad()  # reinitialize the gradient to zero
        #print(nodes.device,edge_index.device)
        prediction=model(nodes, edge_index)
        
        loss=loss_function(prediction,target)
        print(loss.item())
        losses.append(loss.item())
        loss.backward()

        optimizer.step()


/Users/francescosacco/Documents/Language_CA/src/attention.py:122: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525473998/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1615.)
  translation = translation.scatter_reduce(0, receivers.repeat(heads, 1).t(), att, reduce='amax', include_self=False)


11.17861557006836


KeyboardInterrupt: 

: 

In [11]:
from src.graph_initialization import linear_unidirectional_graph_maker
text = "Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. "

print(text, end='')
gpt2_graph_maker = linear_unidirectional_graph_maker(100)
model.most_prob_generate(text, 60, gpt2_graph_maker,temperature=1)


Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.  
The orcs were defeated, but the orcs were not defeated.  
The orcs were defeated, but

KeyboardInterrupt: 

In [20]:
from src.decoder import Loss, last_word_loss
ce = nn.CrossEntropyLoss()

loss_function=Loss(decoder, last_word_loss)

x=torch.randn(10,768)
y=torch.randint(0,50257,(1,))

In [22]:
x=torch.rand(1,20)
y=torch.randint(0,20,(1,))

ce(x,y)

tensor(2.6222)

In [16]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([4, 3, 4])

In [19]:
loss_function(x,y)

RuntimeError: Expected floating point type for target with class probabilities, got Long